In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
try:
    import empyrical as emp
except:
    emp = None
import tushare as ts
import time
import random

from common.log import *
from common.config import Config
from spider.spider_nasdaq import Spider_nasdaq
from spider.spider_coinmarketcap import Spider_coinmarketcap

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

CONF = Config('../conf/secret.yaml').data[0]
ts_token = CONF['TUSHARE']['TOKEN']
ts.set_token(ts_token)
pro = ts.pro_api()

CONF = Config().data[0]
MONGODB = CONF['MONGODB']
NASDAQ = CONF['NASDAQ']
CRYPTOCURRENCY = CONF['CRYPTOCURRENCY']
CRYPTOCURRENCY = list(CRYPTOCURRENCY.keys())
NASDAQ100 = CONF['NASDAQ100']

MONITOR_TARGET = {
    '399300': {'MA': [30, 150], 'ROLLINGMAX': [100], 'ROLLINGMIN': [40], 'NAME': '沪深300(SZ:399300)'},
    '000905': {'MA': [90, 120], 'ROLLINGMAX': [30], 'ROLLINGMIN': [20], 'NAME': '中证500(SH:000905)'},
    '399006': {'MA': [110, 180], 'ROLLINGMAX': [150], 'ROLLINGMIN': [50], 'NAME': '创业板指(SZ:399006)'},
#     'BITCOIN': {'MA': [], 'ROLLINGMAX': [20], 'ROLLINGMIN': [10], 'NAME': 'BITCOIN'},
#     'EOS': {'MA': [], 'ROLLINGMAX': [20], 'ROLLINGMIN': [10], 'NAME': 'EOS'},
}

# MONITOR_TARGET = {
#     '399300': {'MA': [17], 'ROLLINGMAX': [100], 'ROLLINGMIN': [40], 'N': 10, 'NAME': '沪深300(SZ:399300)'},
#     '000905': {'MA': [17], 'ROLLINGMAX': [30], 'ROLLINGMIN': [20], 'N': 10, 'NAME': '中证500(SH:000905)'},
#     '399006': {'MA': [17], 'ROLLINGMAX': [150], 'ROLLINGMIN': [50], 'N': 10, 'NAME': '创业板指(SZ:399006)'},
#     'BITCOIN': {'MA': [5], 'ROLLINGMAX': [20], 'ROLLINGMIN': [10], 'N': 2, 'NAME': 'BITCOIN'},
#     'EOS': {'MA': [5], 'ROLLINGMAX': [20], 'ROLLINGMIN': [10], 'N': 2, 'NAME': 'EOS'},
#     'ETHEREUM': {'MA': [5], 'ROLLINGMAX': [20], 'ROLLINGMIN': [10], 'N': 2, 'NAME': 'ETHEREUM'},
#     'RIPPLE': {'MA': [5], 'ROLLINGMAX': [20], 'ROLLINGMIN': [10], 'N': 2, 'NAME': 'RIPPLE'},
#     'LITECOIN': {'MA': [5], 'ROLLINGMAX': [20], 'ROLLINGMIN': [10], 'N': 2, 'NAME': 'LITECOIN'},
# }

pd

<module 'pandas' from 'd:\\python3\\lib\\site-packages\\pandas\\__init__.py'>

In [2]:
%%time

IS_DOWNLOAD_DATA = True

spider_n = Spider_nasdaq()
spider_c = Spider_coinmarketcap()

for symbol in MONITOR_TARGET:
    if not IS_DOWNLOAD_DATA:
        break
        
    print(symbol)

    # 指数
    if symbol in ['399300', '000905', '000016', '399006']:
        df = ts.get_k_data(code=symbol, index=True, start='2005-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        
    # CRYPTOCURRENCY
    elif symbol in CRYPTOCURRENCY:
        datafile = spider_c.get_coin_data(symbol, start_date='20100101', end_date=time.strftime('%Y%m%d'))
        
    # ETF
    else:
        df = ts.get_k_data(code=symbol, ktype='D', autype='qfq', index=False, start='2005-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        time.sleep(1)

    time.sleep(1)

399300
000905
399006
Wall time: 6.63 s


In [3]:
%%time

stock_df_dict = {}

for symbol in MONITOR_TARGET:
    stock_data_file = '../database/market/%s.csv' % symbol
    try:
        stock_df = pd.read_csv(stock_data_file)
    except:
        print(symbol)
        continue

    # 筛选字段
#     stock_df = stock_df.loc[:, ['date', 'open', 'close']]
    stock_df = stock_df.loc[:, ['date', 'close']]

    # 去掉Nasdaq行情首行的当天行情
    if symbol in NASDAQ100:
        stock_df = stock_df.drop([0])

    # 抛弃空值异常值
    stock_df.dropna(axis=0, how='any', inplace=True)

    # 格式化日期
    # 445 ms ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
    stock_df = stock_df.assign(date=pd.to_datetime(stock_df['date']))  # need .index.to_period('D')

    # 用日期作索引，日期升序排序
    # 95.1 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
    if symbol in NASDAQ100 or symbol in CRYPTOCURRENCY:
        stock_df = stock_df[::-1]
    stock_df.set_index(['date'], inplace=True)
    stock_df.index = stock_df.index.to_period('D')

#     # 计算每天涨跌幅
#     N = MONITOR_TARGET[symbol]['N']
#     stock_df['N_sht'] = stock_df.close.shift(N)
#     stock_df['N_chg'] = (stock_df.close.shift(1) - stock_df.close.shift(N)) / stock_df.close.shift(N)

    for MA_PARAM in MONITOR_TARGET[symbol]['MA']:
        stock_df['MA_%d' % MA_PARAM] = stock_df['close'].rolling(MA_PARAM).mean()
    
    for ROLLINGMAX_PARAM in MONITOR_TARGET[symbol]['ROLLINGMAX']:
        stock_df['ROLLINGMAX_%d' % ROLLINGMAX_PARAM] = stock_df['close'].rolling(ROLLINGMAX_PARAM).max()
        
    for ROLLINGMIN_PARAM in MONITOR_TARGET[symbol]['ROLLINGMIN']:
        stock_df['ROLLINGMIN_%d' % ROLLINGMIN_PARAM] = stock_df['close'].rolling(ROLLINGMIN_PARAM).min()
    
    # 减少数据
    # stock_df = stock_df['2016-01-01':]
    stock_df.dropna(how='any', inplace=True)
    
    stock_df_dict[symbol] = stock_df

Wall time: 92.3 ms


In [4]:
%store MONITOR_TARGET
%store stock_df_dict

Stored 'MONITOR_TARGET' (dict)
Stored 'stock_df_dict' (dict)


In [5]:
for symbol in MONITOR_TARGET:
    symbol
    stock_df_dict[symbol].tail(5)

'399300'

,close,MA_30,MA_150,ROLLINGMAX_100,ROLLINGMIN_40
date,,,,,
2019-05-29,3663.91,3799.412000,3453.190267,4120.61,3583.96
2019-05-30,3641.18,3788.267333,3456.791667,4120.61,3583.96
2019-05-31,3629.79,3773.067333,3460.201933,4120.61,3583.96
2019-06-03,3632.01,3757.893000,3464.119400,4120.61,3583.96
2019-06-04,3598.47,3742.106000,3467.209533,4120.61,3583.96


'000905'

,close,MA_90,MA_120,ROLLINGMAX_30,ROLLINGMIN_20
date,,,,,
2019-05-29,4956.86,5082.956889,4891.179083,5810.18,4841.76
2019-05-30,4926.40,5089.751556,4895.216917,5810.18,4841.76
2019-05-31,4912.00,5096.701000,4899.936083,5810.18,4841.76
2019-06-03,4853.82,5102.328333,4903.912500,5810.18,4841.76
2019-06-04,4795.41,5107.423111,4906.247833,5810.18,4795.41


'399006'

,close,MA_110,MA_180,ROLLINGMAX_150,ROLLINGMIN_50
date,,,,,
2019-05-29,1497.89,1485.482545,1433.488056,1777.26,1443.75
2019-05-30,1485.24,1486.814455,1433.444333,1777.26,1443.75
2019-05-31,1483.66,1488.043182,1433.458056,1777.26,1443.75
2019-06-03,1468.98,1489.484273,1433.519500,1777.26,1443.75
2019-06-04,1456.27,1490.912455,1433.636556,1777.26,1443.75


In [6]:
monitor_df = pd.DataFrame(columns=['SYMBOL', 'DATE', 'NOW', 'BUY', 'BUYDIFF', 'SELL', 'SELLDIFF'])

for symbol in MONITOR_TARGET:
    df = stock_df_dict[symbol].iloc[-40:].copy()
    
    df.reset_index(drop=False, inplace=True)
    # df = df.astype(dtype={'date': 'datetime64[ns]'})
    df['date'] = df['date'].apply(lambda x: x.to_timestamp().to_datetime64())
    df.set_index('date', inplace=True)
    
    today_market = df.iloc[-1]
    now_point = today_market.close
    for col in df.columns:
        if 'ROLLINGMAX' in col:
            buy_point = today_market[col]
            buy_diff = (buy_point - now_point) / now_point * 100
        elif 'ROLLINGMIN' in col:
            sell_point = today_market[col]
            sell_diff = (now_point - sell_point) / now_point * 100
        elif 'MA' in col:
            ma = today_market[col]
            ma_diff = (now_point - ma) / now_point * 100
        elif 'N_chg' in col:
            n_chg = today_market[col] * 100
    
    monitor_df = monitor_df.append({
        'SYMBOL': symbol, 
        'DATE': today_market.name.date(), 
        'NOW': now_point, 
        'BUY': buy_point, 
        'BUYDIFF': '+%.2f%%' % buy_diff, 
        'SELL': sell_point, 
        'SELLDIFF': '-%.2f%%' % sell_diff,
        'MA': ma,
        'MADIFF': '%.2f%%' % ma_diff,
#         'N_sht': today_market.N_sht, 
#         'N_chg': '%.2f%%' % n_chg,
    }, ignore_index=True)
    
    title = '%s, %s, close=%.1f, buy=%.1f(+%.1f%%), sell=%.1f(-%.1f%%)' % \
        (symbol, today_market.name.date(), now_point, buy_point, buy_diff, sell_point, sell_diff)
    
#     df.drop(columns=['N_chg'], inplace=True)
    ax = df.plot(kind='line', title=title, linewidth=0.9, grid=True, figsize=(19, 5))
    ax.yaxis.tick_right()
    
#     display_charts(df, chart_type='stock', kind='line', title=title, figsize=(1000, 600))


monitor_df

ValueError: labels ['N_chg'] not contained in axis

In [ ]:
# df = ts.get_realtime_quotes(['sh', 'sz', 'hs300', 'sz50', 'sh000905', 'zxb', 'cyb'])
# df = ts.get_realtime_quotes(['hs300', 'sh000905', 'cyb'])
# df = ts.get_realtime_quotes(list(MONITOR_TARGET.keys()))
# df
# list(MONITOR_TARGET.keys())